In [90]:
from web3 import Web3
from solcx import compile_source
import pandas as pd
from datetime import datetime

In [2]:
URL="HTTP://127.0.0.1:7545"

In [97]:
def connect(URL):
    web3=Web3(Web3.HTTPProvider(URL))
    print("Connected?:",web3.isConnected())
    return web3

def get_user_accounts():
    return web3.eth.accounts

def create_default_account(account_hash):
    web3.eth.default_account=account_hash
    
def compile_solidity(file):
    with open(file,'r') as f:
        source=f.read()
    return compile_source(source)

def get_abi_bytecode(compiled_sol):
    contract_id,contract_abi=compiled_sol.popitem()
    abi=contract_abi['abi']
    byte_code=contract_abi['bin']
    return abi,byte_code

def deploy_contract(abi,byte_code):
    contract=web3.eth.contract(abi=abi,bytecode=byte_code)
    tx_hash=contract.constructor().transact()
    tx_receipt=web3.eth.waitForTransactionReceipt(tx_hash)
    return tx_receipt

web3=connect(URL)
print(get_user_accounts())

Connected?: True
['0x4cbc87586da52F809D98F122331C06Ba16D6cC3A', '0xA4a4a85a3aA829384e2de7aD2B16B06EBA07A2bA', '0x58325180563bC75A12543FD380dE90048b89700c', '0xB1ec37fA914Fcf087BC1FB394F53292f337150Be', '0x2ad1fD7BD41DBA854523ecf0E3fbef6E7fC91af1', '0x7107246e4d83cF65F0Fbc56584b274B68745f40d', '0xb40518c06eF3dad45D3F35D3e839cF9b4A230f97', '0x464244D4cfDcE7aB54694dbBcd6C3E1ddCAb8c0a', '0x9108B087b51E0fE9Fa9D37eb794386A98050Bf48', '0x426658F34b041dA97E78ae4972Af6f472b1363e4']


In [98]:
create_default_account('0x4cbc87586da52F809D98F122331C06Ba16D6cC3A')

In [114]:
compiled_sol=compile_solidity('contract.sol')

In [115]:
abi,byte_code=get_abi_bytecode(compiled_sol)

In [116]:
abi

[{'inputs': [],
  'name': 'getUser',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'},
   {'internalType': 'uint256', 'name': '', 'type': 'uint256'},
   {'internalType': 'address', 'name': '', 'type': 'address'},
   {'internalType': 'address', 'name': '', 'type': 'address'}],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [{'internalType': 'uint256',
    'name': 'invoiceAmount',
    'type': 'uint256'},
   {'internalType': 'address', 'name': 'to', 'type': 'address'}],
  'name': 'payAmount',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'}]

In [117]:
byte_code

'608060405234801561001057600080fd5b5061035b806100206000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c8063832880e71461003b57806396bc67851461005c575b600080fd5b610043610078565b6040516100539493929190610276565b60405180910390f35b6100766004803603810190610071919061021c565b6100e3565b005b6000806000806000800154600060010154600060020160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16600060030160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16935093509350935090919293565b600042905060008390506000839050600033905060405180608001604052808581526020018481526020018273ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1681525060008082015181600001556020820151816001015560408201518160020160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060608201518160030160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373fffffffffff

In [118]:
receipt=deploy_contract(abi=abi,byte_code=byte_code)

In [119]:
receipt

AttributeDict({'transactionHash': HexBytes('0x4a717da1c52b0b7e7dcd44eae29c525920cdcf2fac70d0f96efb3ee6681f9a5c'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xb02ccf3ac31b5138c32d946baa67452d94bf362eb5461b1bc4904c0d750b8e5f'),
 'blockNumber': 40,
 'from': '0x4cbc87586da52F809D98F122331C06Ba16D6cC3A',
 'to': None,
 'gasUsed': 238622,
 'cumulativeGasUsed': 238622,
 'contractAddress': '0xaF177717125aD2726afc0CA3DB0499B06e61c923',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [120]:
contract=web3.eth.contract(address=receipt['contractAddress'],abi=abi)

In [123]:
contract.functions.payAmount(100,"0x58325180563bC75A12543FD380dE90048b89700c").transact()

HexBytes('0x5e8d4e359f3acd0178aaf6f3191394a44dfa131da16bbdaea174a775998706cf')

In [124]:
data=contract.functions.getUser().call()
data

[1617638508,
 100,
 '0x4cbc87586da52F809D98F122331C06Ba16D6cC3A',
 '0x58325180563bC75A12543FD380dE90048b89700c']

In [125]:
datetime_date=datetime.fromtimestamp(data[0])
pd.to_datetime(datetime_date)

Timestamp('2021-04-05 21:46:48')